In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/clean_data.csv')

In [3]:
df

,Unnamed: 0,week_of,station,station_city,station_province,station_latitude,station_longitude,chart_position,artist_name(s),artist_country,...,language_of_music,visible_ethnic_minority,census_race_classification,artist_gender,m_music,a_artist,p_performance,l_lyrics,artist_is_group,canadian_content
0,0,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,4.0,breakestra,us,...,english,yes,mixed group,male,no,no,no,no,True,False
1,1,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,5.0,candy's .22,us,...,english,no,white,male,no,no,no,no,True,False
2,2,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,7.0,dangerdoom,us,...,english,yes,black,male,no,no,no,no,False,False
3,3,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,8.0,blockhead,us,...,english,no,white,male,no,no,no,no,False,False
4,4,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,9.0,blackalicious,us,...,english,yes,black,male,no,no,no,no,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35853,35854,2010-03-02,ckut,montreal,qc,45.50,-73.58,4.0,kae sun,cdn,...,english,yes,black,male,no,no,no,no,False,False
35854,35855,2010-03-02,ckut,montreal,qc,45.50,-73.58,5.0,rel!g!on,cdn,...,english,yes,black,male,yes,yes,yes,yes,False,True
35855,35856,2010-03-02,ckut,montreal,qc,45.50,-73.58,6.0,swollen members,cdn,...,english,yes,mixed group,male,yes,yes,yes,yes,True,True
35856,35857,2010-03-02,ckut,montreal,qc,45.50,-73.58,7.0,chokeules,cdn,...,english,no,white,male,yes,yes,yes,yes,False,True
